In [1]:
#!pip install nest-asyncio
#!pip install span_marker
#!pip install pypdf
import pandas as pd
import os
import openai
import random
from pathlib import Path
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index.extractors.metadata_extractors import EntityExtractor
from llama_index.node_parser import SentenceSplitter
from llama_index.ingestion import IngestionPipeline
from llama_index import download_loader
import asyncio
import nest_asyncio

In [2]:
# You can also directly access from web 
sbir_df = pd.read_csv('./Downloads/award_data.csv')

/var/folders/fj/31jlj_jj6qq7jtnhvsydh1vr0000gn/T/ipykernel_85178/1286098522.py:1: DtypeWarning: Columns (10,12,13,15,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  sbir_df = pd.read_csv('./Downloads/award_data.csv')


In [3]:
# cleanup and convert to DateTime
sbir_df = sbir_df[~sbir_df['Proposal Award Date'].isna()]
sbir_df['Proposal Award Date'] = pd.to_datetime(sbir_df['Proposal Award Date'])

In [4]:
# limit to records from 2018 to current and to Department of Defence and Department of Energy
sbir_df = sbir_df[sbir_df['Proposal Award Date'] > pd.to_datetime('2018-01-01')]
sbir_df = sbir_df[sbir_df['Agency'].isin(['Department of Defense','Department of Energy'])]

In [5]:
sbir_df

,Company,Award Title,Agency,Branch,Phase,Program,Agency Tracking Number,Contract,Proposal Award Date,Contract End Date,...,Contact Title,Contact Phone,Contact Email,PI Name,PI Title,PI Phone,PI Email,RI Name,RI POC Name,RI POC Phone
2,ADAMAS NANOTECHNOLOGIES INC,Optical NMR using Diamond Quantum Sensing for ...,Department of Energy,NaN,Phase II,SBIR,271112,DE-SC0022441,2023-04-03,04/02/2025,...,NaN,(919) 881-0500,meberle@adamasnano.com,Olga Shenderova,NaN,(919) 881-0500,oshenderova@adamasnano.com,NaN,NaN,NaN
3,"ADDIGURU, LLC",Artificial intelligence and machine learning a...,Department of Defense,Navy,Phase I,SBIR,N222-117-0351,N68335-23-C-0061,2022-11-07,05/09/2023,...,NaN,(504) 858-6357,sk@addiguru.com,Shuchi Khurana,NaN,(504) 858-6357,sk@addiguru.com,NaN,NaN,NaN
4,"ADVANCED CERAMICS MANUFACTURING, LLC",Structural Components with Corrosion Resistant...,Department of Energy,NaN,Phase I,SBIR,270882,DE-SC0023657,2023-02-21,12/20/2023,...,NaN,(520) 547-0861,zwing@acmtucson.com,Zachary Wing,NaN,(520) 547-0861,zwing@acmtucson.com,NaN,NaN,NaN
5,ADVANCED COOLING TECHNOLOGIES INC,Semiconductor Device Simulation Software with ...,Department of Energy,NaN,Phase I,SBIR,270761,DE-SC0023619,2023-02-21,11/20/2023,...,NaN,(717) 205-0602,bill.anderson@1-act.com,Yue Xiao,NaN,(717) 205-0697,yue.xiao@1-act.com,NaN,NaN,NaN
6,ADVANCED COOLING TECHNOLOGIES INC,Rapid and Tunable Cooling Technology for Vacuu...,Department of Energy,NaN,Phase II,SBIR,271181,DE-SC0020508,2023-05-03,05/02/2025,...,NaN,(717) 205-0602,bill.anderson@1-act.com,Patryk Radyjowski,NaN,(717) 205-0603,patryk.radyjowski@1-act.com,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39511,TWELVE BENEFIT CORPORATION,Renewable electricity-powered carbon dioxide c...,Department of Energy,ARPA-E,Phase II,SBIR,1563-1504,DE-AR0000819,2018-12-16,NaN,...,Chief Technology Officer,(650) 291-5614,kendra@opus-12.com,Kendra Kuhl,Chief Technology Officer,(650) 291-5614,kendra@opus-12.com,NaN,NaN,NaN
42244,IR Dynamics,Dynamic IR Window Film to Improve Window Energ...,Department of Energy,ARPA-E,Phase II,SBIR,1429-1517,DE-AR0000745,2018-04-10,NaN,...,NaN,(505) 850-8906,pgclem@sandia.gov,Paul Clem,NaN,(505) 850-8906,pgclem@sandia.gov,NaN,NaN,NaN
44911,"TRITON SYSTEMS, INC.",New Technology for Single Pane Retrofit,Department of Energy,ARPA-E,Phase II,SBIR,1429-1507,DE-AR0000746,2019-02-06,NaN,...,NaN,(978) 856-4141,agavrin@tritonsystems.com,Arthur Gavrin,NaN,(978) 856-4141,agavrin@tritonsystems.com,NaN,NaN,NaN
44929,"TRITON SYSTEMS, INC.",New Technology for Single Pane Retrofit,Department of Energy,ARPA-E,Phase II,SBIR,1429-1507,DE-AR0000746,2018-02-06,NaN,...,NaN,(978) 856-4141,agavrin@tritonsystems.com,Arthur Gavrin,NaN,(978) 856-4141,agavrin@tritonsystems.com,NaN,NaN,NaN


In [7]:
# store records - going forward, we only need to load this
sbir_df.to_csv('./Downloads/SBIR_reduced.csv')

## Create Llamaindex vector index

In [11]:
nest_asyncio.apply()

In [12]:
# Setup OpenAI key
os.environ["OPENAI_API_KEY"] = "<YOUR OPEN API KEY>"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
# load CSV file
SimpleCSVReader = download_loader("SimpleCSVReader")
loader = SimpleCSVReader(encoding="utf-8")
documents = loader.load_data(file=Path('./Downloads/SBIR_reduced.csv'))

In [9]:
# Setup entity extraction pipeline
entity_extractor = EntityExtractor(
    prediction_threshold=0.5,
    label_entities=False,  # include the entity label in the metadata (can be erroneous)
    device="cpu",  # set to "cuda" if you have a GPU
)
node_parser = SentenceSplitter()
transformations = [node_parser, entity_extractor]

In [13]:
# run the entity extraction pipeline on documents (loaded from CSV file)
pipeline = IngestionPipeline(transformations=transformations)
nodes = pipeline.run(documents=documents)

Extracting entities:   0%|          | 0/11473 [00:00<?, ?it/s]

In [14]:
# create word vector 
service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.2)
)

index = VectorStoreIndex(nodes, service_context=service_context)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
# Store the trained context - we don't need to train again,can directly load
index.storage_context.persist(persist_dir="./Downloads/")

In [16]:
# example queries
query_engine = index.as_query_engine()
response = query_engine.query("what are top 5 frequent technical terms")
print(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The top 5 frequent technical terms mentioned in the given context are:
1. 5G
2. Electroplating
3. Wireless Power Transfer (WPT)
4. Deep Learning
5. Race Detection


In [17]:
# example queries
query_engine = index.as_query_engine()
response = query_engine.query("""which company was awarded most $ and how much. This information can be obtained by 
looking into award amount and Company fields""")
print(response)

DT Professional Services LLC was awarded the most amount of money, which was $112,493.


In [18]:
# example queries
query_engine = index.as_query_engine()
response = query_engine.query("""how many awards dis 2PI INC receive?""")
print(response)

Based on the given context information, there is no mention of 2PI INC receiving any awards.


In [22]:
# example queries
query_engine = index.as_query_engine()
response = query_engine.query("""List top 5 companies that were awarded most $ and how much. This information can be obtained by 
looking into award amount and Company fields. Also list technical entities associated with each. """)
print(response)

The top 5 companies that were awarded the most money and their respective award amounts are as follows:

1. Stealth Software Technologies, Inc. - $224,988
   Technical entities associated: Securing Antiquated Languages via program Synthesis and human Assistance (SALSA), Department of Defense, Defense Advanced Research Projects Agency

2. Steam Instruments, Inc. - $199,482
   Technical entities associated: Multimodal Stigmatic Imaging Mass Spectrometry with Flat-Top Laser Probe, Department of Energy

3. DT Professional Services LLC - $112,493
   Technical entities associated: Focus Area 2 - DT Professional's Response to Digital Logistics, Department of Defense, Navy

Please note that the remaining two companies and their award amounts were not provided in the given context.


In [19]:
entities = []
for i in range(len(nodes)):
    #print(node.metadata['entities'])
    if 'entities' not in nodes[i].metadata.keys():
        continue
    for ent in nodes[i].metadata['entities']:
        if ent not in entities:
            entities.append(ent)

In [24]:
with open('Downloads/sbir_entities_llama.txt', 'w') as file:
    # Write each element of the list as a separate line
    for item in entities:
        file.write(f"{item}\n")

In [30]:
# example queries
query_engine = index.as_query_engine()
response = query_engine.query("""List all companies that work that mention Spectometry in their work. Ignore case """)
print(response)

Spectrum Photonics Inc, Specular Theory Inc


In [42]:
# example queries
query_engine = index.as_query_engine()
response = query_engine.query("""How many times 'TRITON SYSTEMS' appears in the records. List one full record""")
print(response)

TRITON SYSTEMS appears three times in the records. Here is one full record:
27670, TRITON SYSTEMS, INC., Extended Wear Deep-Fitting Noise Dosimeter, Department of Defense, Defense Health Program, Phase I, SBIR, H182-002-0021, W81XWH19C0005, 2018-11-05, 06/04/2019, 18.2, 2018.0, 06/20/2018, 06/20/2018, 08/20/2018, DHA182-002, 2019, 149,929, '796010411', N, N, N, 54.0, http://www.tritonsys.com, 200 TURNPIKE ROAD, , CHELMSFORD, MA, 01824, Triton Systems proposes to develop a deep fitting in-the-ear, extended-wear acoustically transparent dosimeter to accurately measure levels and amount of noise exposure near the ear drum., Collette E Jolliffe, , (978) 250-4200, cjolliffe@tritonsystems.com, Brian Fowler, Principal Investigator, (978) 856-4183, bfowler@tritonsystems.com, , ,


In [43]:
# example queries
query_engine = index.as_query_engine()
response = query_engine.query("""return website and address for 'TRITON SYSTEMS'""")
print(response)

The website for TRITON SYSTEMS is http://www.tritonsys.com. The address for TRITON SYSTEMS is 200 TURNPIKE ROAD, CHELMSFORD, MA, 01824.


In [48]:
# example queries
query_engine = index.as_query_engine()
response = query_engine.query("""list all records of TRITON SYSTEMS, INC. """)
print(response)

TRITON SYSTEMS, INC. has the following records:
- Record 27670: Extended Wear Deep-Fitting Noise Dosimeter, Department of Defense, Defense Health Program, Phase I, SBIR
- Record 27671: Expeditionary Metal Additive Manufacturing, Department of Defense, Army, Phase I, SBIR
- Record 27672: Ultra Mobile, Low-shear Casualty Transport Pad for PFC, Department of Defense, Defense Health Program, Phase I, SBIR
- Record 27668: Identification of Open Globe Injuries on the Battlefield, Department of Defense, Defense Health Program, Phase I, SBIR
- Record 27669: Improved Data Tracking System for Crew-Served Weapon Systems, Department of Defense, Navy, Phase I, SBIR


In [32]:
sbir_df.iloc[0]

Company                                                          ADAMAS NANOTECHNOLOGIES INC
Award Title                                Optical NMR using Diamond Quantum Sensing for ...
Agency                                                                  Department of Energy
Branch                                                                                   NaN
Phase                                                                               Phase II
Program                                                                                 SBIR
Agency Tracking Number                                                                271112
Contract                                                                        DE-SC0022441
Proposal Award Date                                                      2023-04-03 00:00:00
Contract End Date                                                                 04/02/2025
Solicitation Number                                                   

In [39]:
sbir_df['Company'].value_counts()

Company
TRITON SYSTEMS, INC.             266
Physical Sciences Inc.           234
Intelligent Automation, Inc.     210
INTELLISENSE SYSTEMS INC         189
CHARLES RIVER ANALYTICS, INC.    171
                                ... 
SWR TECHNOLOGY INC                 1
SYNCHROGRID LTD LLC                1
SYNTROPY SYSTEMS, LLC              1
SYSTEM STRATEGY INC                1
IR Dynamics                        1
Name: count, Length: 4809, dtype: int64

In [54]:
# example queries
query_engine = index.as_query_engine()
response = query_engine.query("""give technical terms associated with 'deep learning' """)
print(response)

Neural nets, deep neural net, deep learning theory, deep learning implementation, deep learning techniques, deep analysis, deep learning integration


In [65]:
nodes[:10]

[TextNode(id_='e8fa0e73-77dc-4331-9a5d-c5538d084090', embedding=None, metadata={'entities': ['Department of Energy']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a9719264-1365-43e2-82a7-91b901943d15', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='106262bb3154c79083556d3121b1e91012b9365aceed62792be006724bfcf109'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='811b09f1-9822-4e3e-bf35-5a30acc6f52a', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='9a720767c80a40beefbc1362fb28298bc6eaf3876566995bdb770f8dad083fee')}, hash='b75ea03ef3ef4a2dd538dbf44279442eccde33715862427f51532b18f5d3faf0', text=", Company, Award Title, Agency, Branch, Phase, Program, Agency Tracking Number, Contract, Proposal Award Date, Contract End Date, Solicitation Number, Solicitation Year, Solicitation Close Date, Proposal Receipt Date, Date of Notification, Topic Code, Award Year, Award Amount, Duns, HU

In [66]:
sbir_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19506 entries, 2 to 45865
Data columns (total 41 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   Company                                  19505 non-null  object        
 1   Award Title                              19506 non-null  object        
 2   Agency                                   19506 non-null  object        
 3   Branch                                   16182 non-null  object        
 4   Phase                                    19506 non-null  object        
 5   Program                                  19506 non-null  object        
 6   Agency Tracking Number                   19506 non-null  object        
 7   Contract                                 19506 non-null  object        
 8   Proposal Award Date                      19506 non-null  datetime64[ns]
 9   Contract End Date                        194

In [67]:
sbir_df.to_csv('./Downloads/sbir_filtered.csv')

In [ ]:
### Graph store instead of vector store - alternate approach

In [ ]:

#from llama_index import KnowledgeGraphIndex
## Takes lot of credits/$$$
'''
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import SimpleGraphStore
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)
kn_index = KnowledgeGraphIndex(nodes, storage_context=storage_context, service_context=service_context)
'''